In [15]:
from collections import defaultdict
responses = []
# define a template string (replace with your actual prompt template)
template = "Prompt for {name} with gender {gender}"

# Provide test_pairs if not already defined (replace with your real data)
if 'test_pairs' not in globals():
    test_pairs = [
        ("Alice", "female"),
        ("Bob", "male"),
        ("Sam", "non-binary"),
    ]

# Provide simple fallbacks so the cell runs even if these helpers are not defined elsewhere.
if 'call_ai' not in globals():
    def call_ai(prompt):
        # minimal stub: return a string that parse_decision understands
        return "yes"  # replace with actual call to your AI service

if 'parse_decision' not in globals():
    def parse_decision(resp):
        # minimal parser for the stubbed response
        return str(resp).strip().lower() in ("yes", "true", "1", "y")

for name, gender in test_pairs:
    prompt = template.format(name=name, gender=gender)
    resp = call_ai(prompt)
    decision = parse_decision(resp)  # True/False
    responses.append((name, gender, decision))

# Aggregate
by_gender = defaultdict(list)
for name, gender, decision in responses:
    by_gender[gender].append(decision)

for g, decs in by_gender.items():
    rate = sum(decs) / len(decs) if decs else float('nan')
    print(g, rate)

female 1.0
male 1.0
non-binary 1.0


In [16]:
def loan_score(income, credit_score, existing_debt, employment_years, loan_amount):
    # inputs must be numeric and validated
    dti = existing_debt / (income + 1e-9)
    score = 0.4 * (credit_score / 900) + 0.4 * (income / (income + loan_amount)) + 0.2 * min(employment_years / 10, 1.0)
    # normalize and return 0-100
    return round(score * 100, 2)

def loan_decision(score, approve_threshold=65.0):
    if score >= approve_threshold:
        return "approve"
    if score >= approve_threshold - 10:
        return "manual_review"
    return "deny"

In [18]:
def test_counterfactual_gender_invariance(call_ai):
    base = {}
    resp_m = call_ai(name='Ram', gender='male', **base)
    resp_f = call_ai(name='Priya', gender='female', **base)
    # Support both object-style responses with .decision and string responses parsed by parse_decision
    if hasattr(resp_m, "decision") and hasattr(resp_f, "decision"):
        decision_m = resp_m.decision
        decision_f = resp_f.decision
    elif 'parse_decision' in globals():
        decision_m = parse_decision(resp_m)
        decision_f = parse_decision(resp_f)
    else:
        decision_m = resp_m
        decision_f = resp_f
    assert decision_m == decision_f